# C1 W1 Group 8

In [1]:
from matplotlib import pyplot as plt
import numpy as np
from PIL import Image, ImageDraw
import seaborn as sns
import pandas as pd
import cv2

from src.data import GT_QSD1_W2_LIST
from src.paths import BBDD_PATH, QSD1_W2_PATH, QSD2_W2_PATH, QST1_W2_PATH, QST2_W2_PATH, WEEK_2_RESULTS_PATH
from src.descriptors import GreyScaleHistogramDescriptor1D, ColorHistogramDescriptor1D, MultiColorSpaceHistogramDescriptor1D
from src.similarities import MSE, L1Distance, ChiSquaredDistance, HistogramIntersection, HellingerKernel, Bhattacharyya
from src.metrics import MeanAveragePrecisionAtK
import pickle
from tqdm import tqdm

## Task 1 - Implement 3D / 2D and block and hierarchical histograms

In [3]:
database_image_PIL_list = [Image.open(db_img_path) for db_img_path in sorted(BBDD_PATH.glob("*.jpg"))]  # Load once
for idx, db_img in enumerate(database_image_PIL_list):
    assert db_img.filename.endswith(f"{idx}.jpg")

In [4]:
def partition_image(image: Image.Image, N: int):
    img_width, img_height = image.size
    
    # Compute info about the partition
    rows = cols = int(N**0.5)  # Assuming N is a perfect square
    part_width = img_width // cols
    part_height = img_height // rows
    
    partitions = []
    
    # Crop each partition
    for row in range(rows):
        for col in range(cols):
            left = col * part_width
            top = row * part_height
            right = left + part_width
            bottom = top + part_height
            part = image.crop((left, top, right, bottom))
            partitions.append(part)
    
    return partitions

def plot_partitions(image, N):
    img_width, img_height = image.size
    
    # Compute info about the partition
    rows = cols = int(N**0.5)  # Assuming N is a perfect square
    part_width = img_width // cols
    part_height = img_height // rows
    
    draw = ImageDraw.Draw(image)
    
    # Vertical partition lines
    for i in range(1, cols):
        x = i * part_width
        draw.line([(x, 0), (x, img_height)], fill="white", width=5)  
    
    # Horizontal partition lines
    for i in range(1, rows):
        y = i * part_height
        draw.line([(0, y), (img_width, y)], fill="white", width=5)  
    
    return image

In [5]:
partitioned_images = {}
partition_levels = [1,4,8]

In [ ]:
for partition_level in partition_levels:
    partitioned_images[partition_level] = [
        partition_image(img, partition_level) 
        for img in tqdm(database_image_PIL_list, desc=f"Partitioning at level {partition_level}")
    ]

In [7]:
descriptors_lab1 = [MultiColorSpaceHistogramDescriptor1D(['HSV', 'LAB', 'YCbCr'], histogram_type='log-chromatic')]

In [ ]:
partitioned_histograms = {}

for descriptor in tqdm(descriptors_lab1, desc="Processing descriptors"):
    partitioned_histograms[descriptor.__class__.__name__] = {}

    for partition_level in tqdm(partition_levels, desc=f"Partition levels for {descriptor.__class__.__name__}", leave=False):
        partitioned_histograms[descriptor.__class__.__name__][partition_level] = []

        for partitions in tqdm(partitioned_images[partition_level]):
            for img in partitions:
                histogram = descriptor.compute(img)
                partitioned_histograms[descriptor.__class__.__name__][partition_level].append(histogram)


## Task 3 - 
For each image in the QS2, remove background using the background color:
- Create a binary mask to evaluate the method
- Compute the descriptors on the foreground pixels

We can see that H is not really meaningful.
Lets see histogram distribution of both S and V